In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pandas as pd

In [ ]:
from transformers import BertTokenizer, BertModel

# Preprocessing and data embeding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset_nlp_project_BIO.csv')

In [ ]:
data

,Misspelled Word,Context,Label (O & N),Output Labels
0,@by5m@|,Such @by5m@| performance today,O,"['S', 'B-O', 'S', 'S']"
1,@d3p7,Your @d3p7 management succeeds,N,"['S', 'B-N', 'S', 'S']"
2,afr1can,why does that afr1can american monkey hold gun...,O,"['S', 'S', 'S', 'B-O', 'S', 'S', 'S', 'S', 'S'..."
3,@ggr355!v3,Your @ggr355!v3 approach worked,N,"['S', 'B-N', 'S', 'S']"
4,@ggr0,Such @ggr0 determination helps,N,"['S', 'B-N', 'S', 'S']"
...,...,...,...,...
1079,wr37ch3d,Such wr37ch3d effort again,O,"['S', 'B-O', 'S', 'S']"
1080,wooshee,I want to see that wooshee's reaction to this ...,O,"['S', 'S', 'S', 'S', 'S', 'B-O', 'S', 'S', 'S'..."
1081,wooshee,I wanna beat that wooshee up,O,"['S', 'S', 'S', 'S', 'B-O', 'S']"
1082,0,score no 0s this time.,N,"['S', 'S', 'B-N', 'S', 'S']"


In [ ]:
# labels_to_indices = {"S": 0, "B-O":1, "B-N":2, "I-O":3,  "I-N": 4, "<pad>": -1}
# indices_to_labels = {0: "S", 1:"B-O", 2:"B-N", 3:"I-O",  4:"I-N", -1: "<pad>"}
labels_to_indices = {"O":0, "N":1}
indices_to_labels = {0: "O", 1:"N"}

In [ ]:
context=  data['Context']
MAX_LENGTH = 40
print(MAX_LENGTH)

context = context.str.lower()

vocab = {"<pad>": 0, '<unk>':1, "<sep>": 2}
for sentence in context:
    tokens = sentence.split()
    for token in tokens:
        if token not in vocab:
            vocab[token] = len(vocab)
vocab_size = len(vocab)
print(vocab_size)
print(vocab)

40
2677
{'<pad>': 0, '<unk>': 1, '<sep>': 2, 'such': 3, '@by5m@|': 4, 'performance': 5, 'today': 6, 'your': 7, '@d3p7': 8, 'management': 9, 'succeeds': 10, 'why': 11, 'does': 12, 'that': 13, 'afr1can': 14, 'american': 15, 'monkey': 16, 'hold': 17, 'gun': 18, 'like': 19, 'that?': 20, '@ggr355!v3': 21, 'approach': 22, 'worked': 23, '@ggr0': 24, 'determination': 25, 'helps': 26, 'problem': 27, 'solving': 28, '@m@z!n6': 29, 'won': 30, 'us': 31, 'the': 32, 'game': 33, '@m@z!ng': 34, 'contribution': 35, 'helped': 36, 'skills': 37, 'showed': 38, 'i': 39, 'heard': 40, 'anna': 41, 'l': 42, 'with': 43, 'depression.': 44, 'he': 45, 'definitely': 46, 'enjoyed': 47, 'session.': 48, 'hate': 49, 'has': 50, 'to': 51, 'be': 52, 'so': 53, 'about': 54, 'this': 55, 'situation.': 56, 'im': 57, 'done': 58, 'bitter': 59, 'bitches': 60, 'its': 61, 'a': 62, 'wrap': 63, 'for': 64, 'that.': 65, 'if': 66, 'you': 67, '@ngry': 68, 'bird': 69, 'theres': 70, 'app': 71, 'getting': 72, '@n6ry': 73, 'at': 74, 'these': 7

In [ ]:
# Set a random seed
random_seed = 42
random.seed(random_seed)

# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset, DataLoader

class OffensiveWordDataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_length):
    self.dataframe = dataframe
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index):
    row = self.dataframe.iloc[index]
    misspelled_word = str(row['Misspelled Word'])
    context = str(row['Context'])
    label = 0 if row['Label (O & N)']=="O" else 1


    inputs = self.tokenizer(misspelled_word, context, add_special_tokens= True,
    return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
    input_ids = inputs['input_ids'].squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label': torch.tensor(label, dtype=torch.long)
    }

In [ ]:
BATCH_SIZE= 16

dataset = OffensiveWordDataset(data, tokenizer, MAX_LENGTH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
from tqdm import tqdm

def generate_embeddings(dataloader, model):
    embeddings = []
    labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc = 'Generating BERT embeddings'):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            label = batch['label']

            outputs = model(input_ids, attention_mask=attention_mask)

            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            embeddings.append(cls_embeddings)
            labels.append(label)
    embeddings = torch.cat(embeddings, dim=0)
    labels = torch.cat(labels, dim=0)

    return embeddings, labels



In [ ]:
embeddings, labels = generate_embeddings(dataloader, bert_model)

torch.save(embeddings, 'bert_embeddings.pt')
torch.save(labels, 'labels.pt')
print("BERT embeddings saved to 'bert_embeddings.pt'")

Generating BERT embeddings: 100%|██████████| 68/68 [01:54<00:00,  1.68s/it]

BERT embeddings saved to 'bert_embeddings.pt'


In [ ]:
from sklearn.model_selection import train_test_split

embeddings = torch.load('bert_embeddings.pt')
labels = torch.load('labels.pt')

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

<ipython-input-31-b723d941102a>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load('bert_embeddings.pt')
<ipython-input-31-b723d941102a>:4: FutureWarni

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:
BATCH_SIZE  = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle = False)

# LSTM test

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_size, num_layers, output_size, dropout):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout = dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        lstm_out, (hidden, _) = self.lstm(x)
        hidden = self.dropout(hidden[-1])
        out = self.fc(hidden)
        return out



In [ ]:
import torch.optim as optim

embedding_dim = 768
hidden_size = 256
num_layers = 1
output_size = 2
dropout = 0.3
EPOCHS = 10

model = LSTMClassifier(embedding_dim, hidden_size, num_layers, output_size, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

def train_model(model, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs.unsqueeze(1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader): .4f}')

train_model(model, train_loader, criterion, optimizer, EPOCHS)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Epoch 1/10, Train Loss:  0.6735
Epoch 2/10, Train Loss:  0.5996
Epoch 3/10, Train Loss:  0.5258
Epoch 4/10, Train Loss:  0.5569
Epoch 5/10, Train Loss:  0.4697
Epoch 6/10, Train Loss:  0.4325
Epoch 7/10, Train Loss:  0.4268
Epoch 8/10, Train Loss:  0.4232
Epoch 9/10, Train Loss:  0.3985
Epoch 10/10, Train Loss:  0.3463


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def test_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch
            outputs = model(inputs.unsqueeze(1))

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print("Test Accuracy:", accuracy_score(all_labels, all_preds))
    print("Classification report: \n", classification_report(all_labels, all_preds))

test_model(model, test_loader)


Test Accuracy: 0.7695852534562212
Classification report: 
               precision    recall  f1-score   support

           0       0.85      0.69      0.77       118
           1       0.70      0.86      0.77        99

    accuracy                           0.77       217
   macro avg       0.78      0.78      0.77       217
weighted avg       0.78      0.77      0.77       217

